# Convert existing PyTorch code to Ray AIR

If you already have working PyTorch code, you don't have to start from scratch to utilize the benefits of Ray AIR. Instead, you can continue to use your existing code and incrementally add Ray AIR components as needed.

Some of the benefits you'll get by using Ray AIR with your existing PyTorch training code:

- Easy distributed data-parallel training on a cluster
- Automatic checkpointing/fault tolerance and result tracking
- Parallel data preprocessing
- Seamless integration with hyperparameter tuning
- Scalable batch prediction
- Scalable model serving

This tutorial will show you how to start with Ray AIR from your existing PyTorch training code. We will learn how to **distribute your training** and do **scalable batch prediction**.


## The example code

The example code we'll be using is that of the [PyTorch quickstart tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html). This code trains a neural network classifier on the FashionMNIST dataset.

You can find the code we used for this tutorial [here on GitHub](https://github.com/pytorch/tutorials/blob/8dddccc4c69116ca724aa82bd5f4596ef7ad119c/beginner_source/basics/quickstart_tutorial.py).

## Unmodified
Let's start with the unmodified code from the example. A thorough explanation of the parts is given in the full tutorial - we'll just focus on the code here.

We start with some imports:

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Then we download the data:

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


/Users/jiaodong/anaconda3/envs/ray/lib/python3.6/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


We can now define the dataloaders:

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

We can then define and instantiate the neural network:

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Define our optimizer and loss:

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

And finally our training loop. Note that we renamed the function from `train` to `train_epoch` to avoid conflicts with the Ray Train module later (which is also called `train`):

In [6]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

And while we're at it, here is our validation loop (note that we sneaked in a `return test_loss` statement and also renamed the function):

In [7]:
def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

Now we can trigger training and save a model:

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_epoch(train_dataloader, model, loss_fn, optimizer)
    test_epoch(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295566  [    0/60000]
loss: 2.291762  [ 6400/60000]
loss: 2.268867  [12800/60000]
loss: 2.262820  [19200/60000]
loss: 2.256001  [25600/60000]
loss: 2.204572  [32000/60000]
loss: 2.225075  [38400/60000]
loss: 2.184233  [44800/60000]
loss: 2.182663  [51200/60000]
loss: 2.154192  [57600/60000]
Test Error: 
 Accuracy: 36.5%, Avg loss: 2.146461 

Epoch 2
-------------------------------
loss: 2.150961  [    0/60000]
loss: 2.147769  [ 6400/60000]
loss: 2.085719  [12800/60000]
loss: 2.107859  [19200/60000]
loss: 2.066872  [25600/60000]
loss: 1.978430  [32000/60000]
loss: 2.029306  [38400/60000]
loss: 1.939256  [44800/60000]
loss: 1.951516  [51200/60000]
loss: 1.881199  [57600/60000]
Test Error: 
 Accuracy: 55.0%, Avg loss: 1.879711 

Epoch 3
-------------------------------
loss: 1.907144  [    0/60000]
loss: 1.879325  [ 6400/60000]
loss: 1.765395  [12800/60000]
loss: 1.815291  [19200/60000]
loss: 1.708041  [25600/60000]
loss: 1.641765  [32000/600

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


We'll cover the rest of the tutorial (loading the model and doing batch prediction) later!

## Introducing a wrapper function (no Ray AIR, yet!)
The notebook-style from the tutorial is great for tutorials, but in your production code you probably wrapped the actual training logic in a function. So let's do this here, too.

Note that we do not add or alter any code here (apart from variable definitions) - we just take the loose bits of code in the current tutorial and put them into one function.

In [10]:
def train_func():
    batch_size = 64
    lr = 1e-3
    epochs = 5
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)
    
    # Get cpu or gpu device for training.
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using {device} device")
    
    model = NeuralNetwork().to(device)
    print(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_epoch(test_dataloader, model, loss_fn)

    print("Done!")

Let's see it in action again:

In [11]:
train_func()

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.311088  [    0/60000]
loss: 2.295296  [ 6400/60000]
loss: 2.271576  [12800/60000]
loss: 2.258537  [19200/60000]
loss: 2.250895  [25600/60000]
loss: 2.216462  [32000/60000]
loss: 2.222296  [38400/60000]
loss: 2.189997  [44800/60000]
loss: 2.188647  [51200/60000]
loss: 2.145895  [57600/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.144711 

Epoch 2
-------------------------------
loss: 2.164661  [    0/60000]
loss: 2.150512  [ 6400/60000]
loss: 2.085597  [12800/60000]
loss: 2.099732  [19200/60000]
loss: 2.047274  [25600/60000]
loss: 1.980986  [32000/60000]
loss: 2.014364  [38400/60000]
loss: 1.930184  [44800

The output should look very similar to the previous ouput.

## Starting with Ray AIR: Distribute the training

As a first step, we want to distribute the training across multiple workers. For this we want to

1. Use data-parallel training by sharding the training data
2. Setup the model to communicate gradient updates across machines
3. Report the results back to Ray Train.


To facilitate this, we only need a few changes to the code:

1. We import Ray Train and Ray AIR Session:

```python
import ray.train as train
from ray.air import session
```


2. We use a `config` dict to configure some hyperparameters (this is not strictly needed but good practice, especially if you want to o hyperparameter tuning later):

```python
def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
```

3. We dynamically adjust the worker batch size according to the number of workers:

```python
    batch_size_per_worker = batch_size // session.get_world_size()
```

4. We prepare the data loader for distributed data sharding:

```python
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
```

5. We prepare the model for distributed gradient updates:

```python
    model = train.torch.prepare_model(model)
```

Note that `train.torch.prepare_model()` also automatically takes care of setting up devices (e.g. GPU training) - so we can get rid of those lines in our current code!


6. We capture the validation loss and report it to Ray train:

```python
        test_loss = test(test_dataloader, model, loss_fn)
        session.report(dict(loss=test_loss))
```

7. In the `train_epoch()` and `test_epoch()` functions we divide the `size` by the world size:

```python
    size = len(dataloader.dataset) // session.get_world_size()  # Divide by word size
```

8. In the `train_epoch()` function we can get rid of the device mapping. Ray Train does this for us:

```python
        # We don't need this anymore! Ray Train does this automatically:
        # X, y = X.to(device), y.to(device) 
```

That's it - you need less than 10 lines of Ray Train-specific code and can otherwise continue to use your original code.

Let's take a look at the resulting code. First the `train_epoch()` function (2 lines changed, and we also commented out the print statement):

In [12]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) // session.get_world_size()  # Divide by word size
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # We don't need this anymore! Ray Train does this automatically:
        # X, y = X.to(device), y.to(device)  

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Then the `test_epoch()` function (1 line changed, and we also commented out the print statement):

In [13]:
def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) // session.get_world_size()  # Divide by word size
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

And lastly, the wrapping `train_func()` where we added 4 lines and modified 2 (apart from the config dict):

In [14]:
import ray.train as train
from ray.air import session

def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
    
    batch_size_per_worker = batch_size // session.get_world_size()
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size_per_worker)
    test_dataloader = DataLoader(test_data, batch_size=batch_size_per_worker)
    
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
    
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_loss = test_epoch(test_dataloader, model, loss_fn)
        session.report(dict(loss=test_loss))

    print("Done!")

Now we'll use Ray Train's TorchTrainer to kick off the training. Note that we can set the hyperparmameters here! In the `scaling_config` we can also configure how many parallel workers to use and if we want to enable GPU training or not.

In [15]:
from ray.train.torch import TorchTrainer


trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={"lr": 1e-3, "batch_size": 64, "epochs": 4},
    scaling_config={"num_workers": 2, "use_gpu": False},
)
result = trainer.fit()
print(f"Last result: {result.metrics}")

2022-07-13 10:38:21,486	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc,iter,total time (s),loss,_timestamp,_time_this_iter_s
TorchTrainer_987e8_00000,TERMINATED,127.0.0.1:65226,4,40.4021,1.21963,1657733950,9.68837


2022-07-13 10:38:26,036	INFO plugin_schema_manager.py:52 -- Loading the default runtime env schemas: ['/Users/jiaodong/Workspace/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/jiaodong/Workspace/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].
2022-07-13 10:38:26,110	WARNING worker.py:1747 -- Warning: The actor TrainTrainable is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(BaseWorkerMixin pid=65290) 2022-07-13 10:38:32,411	INFO config.py:71 -- Setting up process group for: env:// [rank=1, world_size=2]
(BaseWorkerMixin pid=65289) 2022-07-13 10:38:32,411	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=2]
(BaseWorkerMixin pid=65290) [W ProcessGroupGloo.cpp:684] Warning: Unable to resolve hostname to a (local) address. Using the loopback address 

Result for TorchTrainer_987e8_00000:
  _time_this_iter_s: 9.397725820541382
  _timestamp: 1657733922
  _training_iteration: 1
  date: 2022-07-13_10-38-42
  done: false
  experiment_id: 50f32f3aed7d44a69be12b594da40fe6
  hostname: Jiaos-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  loss: 2.155251389096497
  node_ip: 127.0.0.1
  pid: 65226
  time_since_restore: 12.282392978668213
  time_this_iter_s: 12.282392978668213
  time_total_s: 12.282392978668213
  timestamp: 1657733922
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 987e8_00000
  warmup_time: 0.003831148147583008
  
Result for TorchTrainer_987e8_00000:
  _time_this_iter_s: 8.916885137557983
  _timestamp: 1657733931
  _training_iteration: 2
  date: 2022-07-13_10-38-51
  done: false
  experiment_id: 50f32f3aed7d44a69be12b594da40fe6
  hostname: Jiaos-MacBook-Pro-16-inch-2019
  iterations_since_restore: 2
  loss: 1.8652629229673154
  node_ip: 127.0.0.1
  pid: 65226
  time_since_restore: 21.19570803642273
  

2022-07-13 10:39:10,763	INFO tune.py:738 -- Total run time: 47.17 seconds (46.15 seconds for the tuning loop).


Last result: {'loss': 1.219629321508347, '_timestamp': 1657733950, '_time_this_iter_s': 9.688371896743774, '_training_iteration': 4, 'time_this_iter_s': 9.687755107879639, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 4, 'trial_id': '987e8_00000', 'experiment_id': '50f32f3aed7d44a69be12b594da40fe6', 'date': '2022-07-13_10-39-10', 'timestamp': 1657733950, 'time_total_s': 40.40209603309631, 'pid': 65226, 'hostname': 'Jiaos-MacBook-Pro-16-inch-2019', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 40.40209603309631, 'timesteps_since_restore': 0, 'iterations_since_restore': 4, 'warmup_time': 0.003831148147583008, 'experiment_tag': '0'}


Great, this works! You're now training your model in parallel. You could now scale this up to more nodes and workers on your Ray cluster.

But there are a few improvements we can make to the code in order to get the most of the system. For one, we should enable **checkpointing** to get access to the trained model afterwards. Additionally, we should optimize the **data loading** to take place within the workers.

### Enabling checkpointing to retrieve the model
Enabling checkpointing is pretty easy - we just need to pass a `Checkpoint` object with the model state to the `session.report()` API.

```python
    from ray.air import Checkpoint

    checkpoint = Checkpoint.from_dict(
        dict(epoch=t, model=model.module.state_dict())
    )
    session.report(dict(loss=test_loss), checkpoint=checkpoint)
```

Note that the `model.module` part is needed because the model gets wrapped in `torch.nn.DistributedDataParallel` by `train.torch.prepare_model`.

### Move the data loader to the training function

You may have noticed a warning: `Warning: The actor TrainTrainable is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.`.

This is because we load the data outside the training function. Ray then serializes it to make it accessible to the remote tasks (that may get executed on a remote node!). This is not too bad with just 52 MB of data, but imagine this were a full image dataset - you wouldn't want to ship this around the cluster unnecessarily. Instead, you should move the dataset loading part into the `train_func()`. This will then download the data *to disk* once per machine and result in much more efficient data loading.

The result looks like this:

In [16]:
from ray.air import Checkpoint

def load_data():
    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return training_data, test_data


def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
    
    batch_size_per_worker = batch_size // session.get_world_size()
    
    training_data, test_data = load_data()  # <- this is new!
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size_per_worker)
    test_dataloader = DataLoader(test_data, batch_size=batch_size_per_worker)
    
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
    
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_loss = test_epoch(test_dataloader, model, loss_fn)
        checkpoint = Checkpoint.from_dict(
            dict(epoch=t, model=model.module.state_dict())
        )
        session.report(dict(loss=test_loss), checkpoint=checkpoint)

    print("Done!")

Let's train again:

In [17]:
trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={"lr": 1e-3, "batch_size": 64, "epochs": 4},
    scaling_config={"num_workers": 2, "use_gpu": False},
)
result = trainer.fit()


Trial name,status,loc,iter,total time (s),loss,_timestamp,_time_this_iter_s
TorchTrainer_b43fc_00000,TERMINATED,127.0.0.1:65706,4,96.6816,1.21565,1657734050,10.6952


(BaseWorkerMixin pid=65719) 2022-07-13 10:39:16,061	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=2]
(BaseWorkerMixin pid=65719) [W ProcessGroupGloo.cpp:684] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
(BaseWorkerMixin pid=65720) 2022-07-13 10:39:16,061	INFO config.py:71 -- Setting up process group for: env:// [rank=1, world_size=2]
(BaseWorkerMixin pid=65720) [W ProcessGroupGloo.cpp:684] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())


(BaseWorkerMixin pid=65719) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(BaseWorkerMixin pid=65720) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(BaseWorkerMixin pid=65719) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to data/FashionMNIST/raw/train-images-idx3-ubyte.gz
(BaseWorkerMixin pid=65720) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to data/FashionMNIST/raw/train-images-idx3-ubyte.gz


 80%|████████  | 21181440/26421880 [00:34<00:11, 469510.74it/s]
26422272it [00:34, 758920.16it/s]                              
 81%|████████  | 21270528/26421880 [00:34<00:10, 481400.61it/s]


(BaseWorkerMixin pid=65719) Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


...

4422656it [00:04, 1087441.73it/s]                             


(BaseWorkerMixin pid=65720) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
(BaseWorkerMixin pid=65720) Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
(BaseWorkerMixin pid=65720) 


6144it [00:00, 32869647.67it/s]          
(BaseWorkerMixin pid=65720) /Users/jiaodong/anaconda3/envs/ray/lib/python3.6/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
(BaseWorkerMixin pid=65720)   return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
(BaseWorkerMixin pid=65720) 2022-07-13 10:40:08,864	INFO train_loop_utils.py:299 -- Moving model to device: cpu
(BaseWorkerMixin pid=65720) 2022-07-13 10:40:08,865	INFO train_loop_utils.py:343 -- Wrapping provided model in DDP.


Result for TorchTrainer_b43fc_00000:
  _time_this_iter_s: 63.3414740562439
  _timestamp: 1657734019
  _training_iteration: 1
  date: 2022-07-13_10-40-19
  done: false
  experiment_id: 3b3c6e36d57a4e7993aacdbe6cd4c8ed
  hostname: Jiaos-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  loss: 2.138503299397268
  node_ip: 127.0.0.1
  pid: 65706
  should_checkpoint: true
  time_since_restore: 65.89588499069214
  time_this_iter_s: 65.89588499069214
  time_total_s: 65.89588499069214
  timestamp: 1657734019
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b43fc_00000
  warmup_time: 0.0036132335662841797
  
Result for TorchTrainer_b43fc_00000:
  _time_this_iter_s: 10.146686792373657
  _timestamp: 1657734029
  _training_iteration: 2
  date: 2022-07-13_10-40-29
  done: false
  experiment_id: 3b3c6e36d57a4e7993aacdbe6cd4c8ed
  hostname: Jiaos-MacBook-Pro-16-inch-2019
  iterations_since_restore: 2
  loss: 1.8509214698888694
  node_ip: 127.0.0.1
  pid: 65706
  should_checkpoin

2022-07-13 10:40:50,559	INFO tune.py:738 -- Total run time: 99.69 seconds (99.57 seconds for the tuning loop).


We can see our results here:

In [18]:
print(f"Last result: {result.metrics}")
print(f"Checkpoint: {result.checkpoint}")

Last result: {'loss': 1.215654496934004, '_timestamp': 1657734050, '_time_this_iter_s': 10.695234060287476, '_training_iteration': 4, 'time_this_iter_s': 10.697366952896118, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 4, 'trial_id': 'b43fc_00000', 'experiment_id': '3b3c6e36d57a4e7993aacdbe6cd4c8ed', 'date': '2022-07-13_10-40-50', 'timestamp': 1657734050, 'time_total_s': 96.68163204193115, 'pid': 65706, 'hostname': 'Jiaos-MacBook-Pro-16-inch-2019', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 96.68163204193115, 'timesteps_since_restore': 0, 'iterations_since_restore': 4, 'warmup_time': 0.0036132335662841797, 'experiment_tag': '0'}
Checkpoint: <ray.air.checkpoint.Checkpoint object at 0x7f9dc87fab38>


## Loading the model for prediction
You may have noticed that we skipped one part of the original tutorial - loading the model and using it for inference. The original code looks like this (we've wrapped it in a function):

In [19]:
def predict_from_model(model):
    classes = [
        "T-shirt/top",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ]

    model.eval()
    x, y = test_data[0][0], test_data[0][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')


We can use our saved model with the existing code to do prediction:

In [20]:
from ray.train.torch import load_checkpoint

model, _ = load_checkpoint(result.checkpoint, NeuralNetwork())

predict_from_model(model)

Predicted: "Ankle boot", Actual: "Ankle boot"


To predict more than one example, we can use a loop:

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

def predict_from_model(model, data):
    model.eval()
    with torch.no_grad():
        for x, y in data:
            pred = model(x)
            predicted, actual = classes[pred[0].argmax(0)], classes[y]
            print(f'Predicted: "{predicted}", Actual: "{actual}"')


In [22]:
predict_from_model(model, [test_data[i] for i in range(10)])

Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Shirt"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Coat", Actual: "Coat"
Predicted: "Pullover", Actual: "Shirt"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"


## Using Ray AIR for scalable batch prediction
However, we can also use Ray AIRs `BatchPredictor` class to do scalable prediction.

In [24]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.torch import TorchPredictor

batch_predictor = BatchPredictor.from_checkpoint(result.checkpoint, TorchPredictor, model=NeuralNetwork())

Batch predictors work with Ray Datasets. Here we convert our test dataset into a Ray Dataset - note that this is not very efficient, and you can look at our {ref}`other tutorials <air-examples-ref>` to see more efficient ways to generate a Ray Dataset.

In [25]:
import ray.data

ds = ray.data.from_items([x.numpy() for x, y in test_data])

We can then trigger prediction with two workers:

In [26]:
results = batch_predictor.predict(ds, min_scoring_workers=2)

Map Progress (2 actors 1 pending): 100%|██████████| 200/200 [00:02<00:00, 70.01it/s]


`results` is another Ray Dataset. We can use `results.show()` to see our prediction results:

In [27]:
results.show()

{'predictions': array([-1.6813023 , -1.80252   , -0.7062941 , -1.311813  , -0.73570144,
        1.5710734 , -0.7933277 ,  2.0013504 ,  1.3654878 ,  2.3410547 ],
      dtype=float32)}
{'predictions': array([ 0.7655406 , -2.3314183 ,  2.7599745 , -0.9481916 ,  2.381936  ,
       -1.7827132 ,  1.9278868 , -3.1977224 ,  0.99582016, -1.4932251 ],
      dtype=float32)}
{'predictions': array([ 1.3619348 ,  3.6063552 , -0.31104898,  2.543014  ,  0.35176522,
       -2.2156405 ,  0.33978355, -2.346588  , -1.7794112 , -2.3220763 ],
      dtype=float32)}
{'predictions': array([ 1.0049653 ,  2.828181  , -0.29254377,  2.0342605 ,  0.12778719,
       -1.6141529 ,  0.17694427, -1.7565594 , -1.4074212 , -1.6818824 ],
      dtype=float32)}
{'predictions': array([ 0.7404187 , -1.0129585 ,  1.0854365 , -0.20976087,  1.0174558 ,
       -0.9567458 ,  1.0075954 , -1.7656276 ,  0.42417505, -0.82513285],
      dtype=float32)}
{'predictions': array([ 1.4985809 ,  2.499547  ,  0.12339873,  1.9594493 ,  0.717446 

If we want to convert these predictions into class names (as in the original example), we can use a `map` function to do this:

In [29]:
predicted_classes = results.map_batches(
    lambda batch: [classes[pred.to_numpy().argmax(0)] for pred in batch["predictions"]], 
    batch_format="pandas")

Map_Batches: 100%|██████████| 200/200 [00:02<00:00, 80.05it/s]


To compare this with the actual labels, let's create a Ray dataset for these and zip it together with the predicted classes:

In [30]:
real_classes = ray.data.from_items([classes[y] for x, y in test_data])
merged = predicted_classes.zip(real_classes)

Let's examine our results:

In [31]:
merged.show()

('Ankle boot', 'Ankle boot')
('Pullover', 'Pullover')
('Trouser', 'Trouser')
('Trouser', 'Trouser')
('Pullover', 'Shirt')
('Trouser', 'Trouser')
('Coat', 'Coat')
('Pullover', 'Shirt')
('Sneaker', 'Sandal')
('Sneaker', 'Sneaker')
('Pullover', 'Coat')
('Ankle boot', 'Sandal')
('Sneaker', 'Sneaker')
('Dress', 'Dress')
('Coat', 'Coat')
('Trouser', 'Trouser')
('Pullover', 'Pullover')
('Pullover', 'Coat')
('Bag', 'Bag')
('T-shirt/top', 'T-shirt/top')


## Summary

This tutorial demonstrated how to turn your existing PyTorch code into code you can use with Ray AIR.

We learned how to
- enable distributed training using Ray Train abstractions
- save and retrieve model checkpoints via Ray AIR
- load a model for batch prediction

In our {ref}`other examples <air-examples-ref>` you can learn how to do more things with the Ray AIR API, such as **serving your model with Ray Serve** or **tune your hyperparameters with Ray Tune.** You can also learn how to **construct Ray Datasets** to leverage Ray AIR's **preprocessing** API.

We hope this tutorial gave you a good starting point to leverage Ray AIR. If you have any questions, suggestions, or run into any problems pelase reach out on [Discuss](https://discuss.ray.io/) or [GitHub](https://github.com/ray-project/ray)!